## Imports and Constants

In [ ]:
import numpy as np
import scipy
import cv2
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

encoder_path = '../models/encoder'
decoder_path = '../models/decoder'
autoencoder_path = '../models/autoencoder'

## Load and process the MNIST dataset

In [27]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255
# Build a Validation dataset from both testing and training images
(val_images, val_labels) = (np.concatenate((train_images[0:5000], test_images[0:5000])), np.concatenate((train_labels[0:5000], test_labels[0:5000])))
# Build the Testing and Training dataset
(train_images, train_labels) = (train_images[5000:], train_labels[5000:])
(test_images, test_labels) = (test_images[5000:], test_labels[5000:])
# Add a (naive) variation parameter in the targets
def add_var_parameter(dataset):
  ohe = np.zeros((dataset.shape[0], 10)) # The one-hot encoding version of the target
  var = np.zeros((dataset.shape[0], 1)) # The variation parameter
  for i in range(dataset.shape[0]):
    ohe[i, dataset[i]] = 1
    var[i] = i/dataset.shape[0]
  return ohe, var
# Do for each dataset
train_ohe, train_var = add_var_parameter(train_labels)
val_ohe, val_var = add_var_parameter(val_labels)
test_ohe, test_var = add_var_parameter(test_labels)

## Create the autoencoder

### Encoder

In [28]:
encoder_input = layers.Input(shape = (28, 28, 1), name='Input')
x = layers.Conv2D(64, (3, 3), activation='sigmoid', padding='same', name='Layer1_nt_')(encoder_input)
x = layers.MaxPool2D((2, 2), name='Layer2_nt_')(x)
x = layers.BatchNormalization(name='Layer3_nt_')(x)
x = layers.Conv2D(96, (3, 3), activation='sigmoid', padding='same', name='Layer4_nt_')(x)
x = layers.BatchNormalization(name='Layer5_nt_')(x)
x = layers.MaxPool2D((2, 2), name='Layer6_nt_')(x)
x = layers.Conv2D(128, (3, 3), activation='sigmoid', padding='same', name='Layer7_nt_')(x)
x = layers.BatchNormalization(name='Layer8_nt_')(x)
x = layers.Flatten(name='Layer9')(x)
encoder_out1 = layers.Dense(10, activation='softmax', name='Output1_nt_')(x)
x = layers.LayerNormalization(name='Layer10')(x)
x = layers.Dense(128, activation='sigmoid', name='Layer11')(x)
encoder_out2 = layers.Dense(1, activation='sigmoid', name='Output2')(x)
encoder = models.Model(inputs=encoder_input, outputs=[encoder_out1, encoder_out2])
tf.keras.utils.plot_model(encoder, show_shapes=True, to_file='../models/encoder.png')
encoder.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
Layer1_nt_ (Conv2D)             (None, 28, 28, 64)   640         Input[0][0]                      
__________________________________________________________________________________________________
Layer2_nt_ (MaxPooling2D)       (None, 14, 14, 64)   0           Layer1_nt_[0][0]                 
__________________________________________________________________________________________________
Layer3_nt_ (BatchNormalization) (None, 14, 14, 64)   256         Layer2_nt_[0][0]                 
___________________________________________________________________________________________

### Decoder

In [29]:
decoder_input1 = layers.Input(shape = (10,), name='Input1')
decoder_input2 = layers.Input(shape = (1,), name='Input2')
x = layers.Dense(128, activation='sigmoid', name='Layer1')(decoder_input2)
x = layers.LayerNormalization(name='Layer2')(x)
x = layers.Concatenate(axis=1, name='Layer3')([decoder_input1, x])
x = layers.Dense(6272, activation='sigmoid', name='Layer4')(x)
x = layers.Reshape((7, 7, 128), name='Layer5')(x)
x = layers.BatchNormalization(name='Layer6')(x)
x = layers.Conv2D(128, (3, 3), activation='sigmoid', padding='same', name='Layer7')(x)
x = layers.BatchNormalization(name='Layer8')(x)
x = layers.Conv2D(96, (3, 3), activation='sigmoid', padding='same', name='Layer9')(x)
x = layers.BatchNormalization(name='Layer10')(x)
x = layers.UpSampling2D((2, 2), name='Layer11')(x)
x = layers.Conv2D(64, (3, 3), activation='sigmoid', padding='same', name='Layer12')(x)
x = layers.BatchNormalization(name='Layer13')(x)
x = layers.UpSampling2D((2, 2), name='Layer14')(x)
x = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='Layer15')(x)
x = layers.Reshape((28, 28), name='Output')(x)
decoder = models.Model(inputs=[decoder_input1, decoder_input2], outputs=x)
tf.keras.utils.plot_model(decoder, show_shapes=True, to_file='../models/decoder.png')
decoder.summary()


Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input2 (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
Layer1 (Dense)                  (None, 128)          256         Input2[0][0]                     
__________________________________________________________________________________________________
Input1 (InputLayer)             [(None, 10)]         0                                            
__________________________________________________________________________________________________
Layer2 (LayerNormalization)     (None, 128)          256         Layer1[0][0]                     
___________________________________________________________________________________________

### Autoencoder

In [30]:
ae_input = layers.Input(shape = (28, 28))
latent_vector = encoder(ae_input)
ae_output = decoder(latent_vector)
autoencoder = models.Model(inputs = ae_input , outputs = ae_output)
tf.keras.utils.plot_model(autoencoder, show_shapes=True, to_file='../models/autoencoder.png')
autoencoder.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
model_11 (Functional)           [(None, 10), (None,  1046251     input_4[0][0]                    
__________________________________________________________________________________________________
model_12 (Functional)           (None, 28, 28)       1188193     model_11[0][0]                   
                                                                 model_11[0][1]                   
Total params: 2,234,444
Trainable params: 2,233,036
Non-trainable params: 1,408
__________________________________________________________________________________________________


## Training the autoencoder

### Training the encoder - classifier

In [ ]:
# Create a dummy loss for the first training - we don't care about the variation parameter for now
def dum_loss(y_true, y_pred):
  return 0

# Compile the encoder
encoder.compile(optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9), loss = {'Output1_nt_': 'categorical_crossentropy', 'Output2': dum_loss}, metrics=['acc'])
# Train it with the training set example and encoded labels
encoder.fit(train_images, [train_ohe, train_var], epochs=3, validation_data=(val_images, [val_ohe, val_var]))

In [32]:
# Fix encoder weights that correspond to the classifier part
for layer in encoder.layers:
  if '_nt_' in layer.name:
    layer.trainable = False
    print('Fixed weights for {}'.format(layer.name))

Fixed weights for Layer1_nt_
Fixed weights for Layer2_nt_
Fixed weights for Layer3_nt_
Fixed weights for Layer4_nt_
Fixed weights for Layer5_nt_
Fixed weights for Layer6_nt_
Fixed weights for Layer7_nt_
Fixed weights for Layer8_nt_
Fixed weights for Output1_nt_


### Pre-training the decoder

In [ ]:
# Compile the decoder
decoder.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss = "binary_crossentropy")
# For performance purposes, we train the decoder a few iterations so that the weights are already initialized in a good way
# We train it giving the target in input and the image matrix in output
decoder.fit([train_ohe, train_var], train_images, batch_size=512, epochs=15, validation_data=([val_ohe, val_var], val_images))

### Training the whole autoencoder

In [ ]:
def random_decode():
  ohe = np.zeros((1, 10)) # One hot encoded vector init
  ohe[0, np.random.random_integers(0, 9)] = 1 # Random number
  var = np.zeros((1, 1)) # Variation parameter init
  var[0] = np.random.random_integers(0, 100)/100 # Random ratio
  image = decoder.predict([ohe, var])[0]
  plt.rcParams["figure.figsize"] = (1,1)
  plt.imshow(image, cmap = "gray")
  plt.show()
# Compile the autoencoder
autoencoder.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss = "binary_crossentropy")
# Train the whole autoencoder
for i in range(25):
  random_decode()
  # We train one iteration by one to see the changes
  autoencoder.fit(train_images, train_images, batch_size=128, epochs=1, validation_data=[val_images, val_images])

### Saving the model weights

In [ ]:
encoder.save(encoder_path)
decoder.save(decoder_path)
autoencoder.save(autoencoder_path)

INFO:tensorflow:Assets written to: drive/My Drive/IIT/CV/P2V2/models/encoder/assets


## Results

### Loading the models

In [ ]:
encoder = tf.keras.models.load_model(encoder_path, custom_objects={'dum_loss': dum_loss})
decoder = tf.keras.models.load_model(decoder_path)
autoencoder = tf.keras.models.load_model(autoencoder_path)

### VIDEO - Transitions from number to number for multiple variations

In [346]:
# 1. Generate the frames
frames = 60 # Number of frames per number
col = 32 # Number of variations (vertically)
row = 32 # Number of variations (horizontally)
nb_of_var = col*row # Total number of variations
# Initialize the input vectors
images_ohe = np.zeros((frames*10*nb_of_var, 10)) # 'One hot encoding' vectors
images_var = np.zeros((frames*10*nb_of_var, 1)) # Variation vector

for i in range(images_var.shape[0]):
  images_var[i] = (i/nb_of_var)%1 # For each frame, compute the variation parameter (from 0 (top left corner) to 1 (bottom right corner))

for n in range(10): # For each number
  for i in range(frames): # For each frame per number
    for v in range(nb_of_var): # For each variation
      progress = i/frames # Compute the progress of the transition
      progress = (np.tanh(10*(progress-0.5))+1)/2 # Curve the progress (ease in - ease out)
      image_i = (n*frames+i)*nb_of_var + v # Compute the index of the frame
      images_ohe[image_i, n%10] = 1 - progress # Add the value for the current number in the OHE vector of the frame
      images_ohe[image_i, (n+1)%10] = progress # Add the value for the next number in the OHE vector of the frame

images = decoder.predict([images_ohe, images_var]) # Decode each frame with the decoder
images = images.reshape((frames*10, row, col, 28, 28)) # Reshape the images matrix

# 2. Create the video
video_name = 'results/Number2Number.mp4' # Video path
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 25, (28*row,28*col))
for f in range(images.shape[0]): # For each frame
  im = np.zeros((28*row, 28*col, 3)) # Create a zero matrix
  for r in range(images.shape[1]): # For each row
    for c in range(images.shape[2]): # For each column
      # Draw the image in the matrix
      im[r*28:r*28+28,c*28:c*28+28, 0] = images[f, r, c]*255
      im[r*28:r*28+28,c*28:c*28+28, 1] = images[f, r, c]*255
      im[r*28:r*28+28,c*28:c*28+28, 2] = images[f, r, c]*255
  # Write the frame in the video
  video.write(im.astype('uint8'))
# Save the video
cv2.destroyAllWindows()
video.release()

### VIDEO - Transitions from number to random number along with continuous variation changes

In [348]:
# 1. Generate frames
number_transition_frames = 60 # Number of frame for each number transition
variation_transition_frames = 250 # Number of frame for a whole variation transition (from 0 to 1 - or 1 to 0)
number_transitions = 64 # Number of transitions to complete
total_frames = (number_transition_frames)*number_transitions # Compute the total number of frames
images_ohe = np.zeros((total_frames, 10)) # 'One hot encoding' vectors
images_var = np.zeros((total_frames, 1)) # Variation vector

current_n = 0 # Current number
image_i = 0 # Current frame index
for nt in range(number_transitions):
  last_n = current_n
  # Chose a different number randomly
  while current_n == last_n:
    current_n = np.random.random_integers(0, 9)
  # Do a transition to this new number
  for ntf in range(number_transition_frames): # For each transition frame
    progress = ntf/number_transition_frames # Compute the transition progress
    progress = (np.tanh(10*(progress-0.5))+1)/2 # Curve the transition (ease in - ease out)
    images_ohe[image_i, last_n%10] = 1 - progress # Add the value for the current number in the OHE vector of the frame
    images_ohe[image_i, current_n%10] = progress # Add the value for the next number in the OHE vector of the frame
    images_var[image_i] = (np.cos(image_i/variation_transition_frames*np.pi)+1)/2 # Add the variation parameter (cosinusoide curve from 0 to 1 and 1 to 0)
    image_i+=1

images = decoder.predict([images_ohe, images_var]) # Decode each frame with the decoder

# 2. Create the video
# Create an information header
header_height = 32
bar_height = 6
space = 8
frame_size = 512
# Create the video file
video_name = 'results/RandomTransformations.mp4' # Video path
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 25, (frame_size,frame_size+header_height))

for f in range(total_frames): # For each frame
  im = np.zeros((28, 28, 3)) # Create the number image matrix
  # Draw the image in the matrix
  im[:,:, 0] = images[f]*255
  im[:,:, 1] = images[f]*255
  im[:,:, 2] = images[f]*255
  # Create the frame matrix
  out = np.zeros((frame_size+header_height,frame_size, 3))
  # Resize the number image
  resized = cv2.resize(im, (frame_size, frame_size), fx=0,fy=0, interpolation = cv2.INTER_LINEAR)
  # Fill the frame with the image
  out[header_height:, :, :] = resized
  # Add the information header
  max_y = lambda x: space + int((x)*(frame_size - space*2))
  var_max_y = max_y(images_var[f])
  num_max_y = max_y(sum(images_ohe[f]*[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])/9)
  out[space:space+bar_height, space:var_max_y, 0] = 255
  out[2*space+bar_height:2*space+2*bar_height, space:num_max_y, 2] = 255
  # Write the frame
  video.write(out.astype('uint8'))
# Save the video
cv2.destroyAllWindows()
video.release()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: This function is deprecated. Please call randint(0, 9 + 1) instead
  from ipykernel import kernelapp as app


### IMAGE - Transformation between ordered numbers VS variation transformations

In [350]:
# 1. Generate the images
# Image parameters
col = 64
row = 64
nb_of_var = col*row
# Generate the images
images_ohe = np.zeros((nb_of_var, 10))
images_var = np.zeros((nb_of_var, 1))
for i in range(nb_of_var):
  images_var[i] = (i/row)%1 # Compute the variation parameter for each number (0 to 1 from the top to the bottom of the image)

for r in range(row): # For each row
  for c in range(col): # For each column
    progress = c/col * 10.0 # At what number we are, from 0 to 9
    n = int(progress) # Compute the actual integer
    image_i = c*row + r # Compute the image index
    images_ohe[image_i, n%10] = 1 - (progress - n) # Compute the OHE participation for n
    images_ohe[image_i, (n+1)%10] = (progress - n) # Compute the OHE participation for n+1

images = decoder.predict([images_ohe, images_var]) # Use the decoder to generate the images
images = images.reshape((col, row, 28, 28)) # Reshape the images imatrix

# 2. Plot the images
im = np.zeros((28*row, 28*col)) # Initialize the matrix
for c in range(col): # For each column
  for r in range(row): # For each row
    im[r*28:r*28+28,c*28:c*28+28] = images[c, r] # Add the image at the right position
# Show and save the figure
plt.rcParams["figure.figsize"] = (col,row)
plt.imshow(im, cmap = "gray")
plt.axis("off")
plt.savefig(f'out.png')
plt.show()